In [1]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import chi2

In [2]:
def IC(loss_by_K, changepoints, g_index, N, T, K, dfs, q = 1.96, C=0, Kl_fun='log(NT)/T'):

    loss = 0
    for k in range(K):
        loss += loss_by_K[k]

    if Kl_fun == 'log(NT)/T':
        Kl = C * K * np.log(N*T)/T
    elif Kl_fun == 'chi2':
        # dat_out = pd.DataFrame({'cluster': g_index, 'changepoint': changepoints})
        # print('g_index =', g_index)
        cluster_cp = pd.crosstab(g_index, changepoints)
        cols = cluster_cp.columns
        bt = cluster_cp.apply(lambda x: x > 0)
        cluster_cp = np.concatenate(bt.apply(lambda x: list(cols[x.values]), axis=1))
        _, counts = np.unique(g_index, return_counts=True)

        n_timepoint_after_cp = T - 1 - cluster_cp
        # dim_covariate = 24
        Kl = 0
        for k in range(K):
            dim_covariate = dfs[k]
            chi2_q = chi2.ppf(0.9, dim_covariate)  # dfs[k]
            sigma_k = -loss_by_K[k] / (n_timepoint_after_cp[k] * counts[k] - dim_covariate)
            print("sigma_k =", sigma_k)
            print("dim_covariate =", dim_covariate)
            Kl += sigma_k * chi2_q / n_timepoint_after_cp[k] + 2*q * np.sqrt(counts[k] / n_timepoint_after_cp[k])
    elif Kl_fun == '0':
        Kl = 0

    # print("kl", Kl)
    _, indicesList, occurCount = np.unique(g_index, return_index=True, return_counts=True)
    # print('c',occurCount.dot((T-1 -changepoints)[np.s_[indicesList]])/(N*T) * C * np.log(np.sum(T-1 -changepoints)))

    print('loss =', loss)
    ic = loss + Kl
    print("Kl =", Kl)
    print("IC =", ic)
    return ic

In [3]:
# number of individuals
N = 477
# number of time points (starting from 0, to 26 weeks)
T = 27
# the candidate values of K
Ks = [1,2,3,4,5,6,7,8]
file_name = "result_with_llk.dat"
out = pickle.load(open(file_name, "rb"))

In [4]:
# obtain change points under K clusters
changepoints = out['changepoints_all']
# obtain cluster indices under K clusters
clusters = out['clusters_all']
# obtain unnormalized loglikelihood
loss_unnormalized = out['loss_unnormalized_all']
# obtain normalized loglikelihood (unnomalized llk divided by tau_k)
loss_normalized = out['loss_normalized_all']
# obtain the dimension of nonzero coefficients in each cluster
dfs = out['df_all']
# obtain the residual variances
sigma = out['sigma_all']

In [59]:
loss_n = np.zeros(8)
loss_un = np.zeros(8)
cluster_cp = [None]*8
cluster_no = [None]*8
sigma2 = [None]*8
for K in Ks:
    loss_n[K-1] += sum(loss_normalized[K])
    loss_un[K-1] += sum(loss_unnormalized[K])
    cp = pd.crosstab(clusters[K], changepoints[K])
    cols = cp.columns
    bt = cp.apply(lambda x: x > 0)
    cluster_cp[K-1] = T-1-np.concatenate(bt.apply(lambda x: list(cols[x.values]), axis=1))
    _, counts = np.unique(clusters[K], return_counts=True)
    cluster_no[K-1] = counts
    sigma2[K-1] = -loss_unnormalized[K]/(cluster_cp[K-1]*cluster_no[K-1]-dfs[K])

In [64]:
loss_n

array([-879.71096842, -823.70343601, -799.25199135, -795.00684885,
       -774.67011779, -766.82077919, -758.64049004, -754.91529528])

In [65]:
sigma2

[array([1.846044]),
 array([1.60898807, 1.8577468 ]),
 array([1.60580057, 1.90504156, 1.55393718]),
 array([1.47517953, 1.56293155, 1.90000459, 1.72529724]),
 array([1.5293257 , 1.82458507, 1.37245868, 1.77166491, 1.86073247]),
 array([1.51709018, 1.78664353, 1.32383533, 1.63597109, 1.85039038,
        1.75351954]),
 array([1.52209132, 1.84289623, 1.32099281, 1.70182672, 1.94283255,
        1.5472716 , 1.41111229]),
 array([1.59046957, 1.64317262, 2.13088241, 1.46448779, 1.51527116,
        1.31441286, 1.57179521, 1.85009468])]

In [66]:
cluster_cp

[array([13]),
 array([12, 20]),
 array([16, 26, 20]),
 array([26, 17, 26, 12]),
 array([21, 26, 26, 26, 26]),
 array([26, 26, 26, 26, 26, 26]),
 array([26, 26, 26, 26, 26, 26, 26]),
 array([26, 26, 26, 26, 26, 26, 26, 26])]

In [67]:
cluster_no

[array([477]),
 array([245, 232]),
 array([132, 154, 191]),
 array([ 84, 159, 133, 101]),
 array([137,  89, 110,  79,  62]),
 array([131,  85,  80,  68,  51,  62]),
 array([79, 79, 76, 65, 47, 74, 57]),
 array([43, 58, 23, 57, 69, 78, 68, 81])]

In [71]:
penalty_n = [None]*8
penalty_ns = np.zeros(8)
for K in Ks:
    penalty_n[K-1] = sigma2[K-1] * np.sqrt(cluster_no[K-1]/cluster_cp[K-1])
    penalty_ns[K-1] = np.sum(penalty_n[K-1])

In [82]:
penalty_ns

array([11.18225938, 13.59743897, 14.05082519, 16.73401181, 16.06652154,
       16.90303272, 18.12673278, 19.22467731])

In [75]:
loss_n - penalty_ns*2*1.96

array([-923.54542519, -877.00539676, -854.3312261 , -860.60417514,
       -837.65088225, -833.08066747, -829.69728254, -830.27603034])

In [76]:
loss_n - 

array([-879.71096842, -823.70343601, -799.25199135, -795.00684885,
       -774.67011779, -766.82077919, -758.64049004, -754.91529528])

In [77]:
cluster_ns = np.zeros(8)
for K in Ks: 
    cluster_ns[K-1] = np.sum(np.sqrt(cluster_no[K-1]))

In [78]:
cluster_ns

array([21.84032967, 30.88402205, 37.7190739 , 43.35710982, 48.38897182,
       52.87098706, 57.56425877, 60.90347357])

In [99]:
loss_n - range(K)*np.sqrt(N)

array([-879.71096842, -845.54376568, -842.93265069, -860.52783785,
       -862.03143646, -876.02242753, -889.68246804, -907.79760295])

In [86]:
clusters

{1: array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [89]:
len(clusters[1])

477